<a href="https://colab.research.google.com/github/Stitaprajna/Practise-codes/blob/main/Chittal_Technologies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_ta
!pip install yfinance
import pandas_ta as ta
import yfinance as yf
import numpy as np
from datetime import date, datetime, time, timedelta
import pytz
import pandas as pd
from time import sleep
import traceback
import requests

def squeeze_momentum(df):
  # parameter setup (default values in the original indicator)
  length = 20
  mult = 2
  length_KC = 20
  mult_KC = 1.5

  # calculate Bollinger Bands
  # moving average
  m_avg = df['close'].rolling(window=length).mean()
  # standard deviation
  m_std = df['close'].rolling(window=length).std(ddof=0)
  # upper Bollinger Bands
  df['upper_BB'] = m_avg + mult * m_std
  # lower Bollinger Bands
  df['lower_BB'] = m_avg - mult * m_std

  # calculate Keltner Channel
  # first we need to calculate True Range
  df['tr0'] = abs(df["high"] - df["low"])
  df['tr1'] = abs(df["high"] - df["close"].shift())
  df['tr2'] = abs(df["low"] - df["close"].shift())
  df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
  # moving average of the TR
  range_ma = df['tr'].rolling(window=length_KC).mean()
  # upper Keltner Channel
  df['upper_KC'] = m_avg + range_ma * mult_KC
  # lower Keltner Channel
  df['lower_KC'] = m_avg - range_ma * mult_KC

  # calculate bar value
  highest = df['high'].rolling(window = length_KC).max()
  lowest = df['low'].rolling(window = length_KC).min()
  m1 = (highest + lowest)/2
  df['value'] = (df['close'] - (m1 + m_avg)/2)
  fit_y = np.array(range(0,length_KC))
  df['value'] = df['value'].rolling(window = length_KC).apply(lambda x:
                            np.polyfit(fit_y, x, 1)[0] * (length_KC-1) +
                            np.polyfit(fit_y, x, 1)[1], raw=True)
  # df['value'] = df['value'] * 100 / df['value'].max()
  # check for 'squeeze'
  df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
  df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])
  return df


def support_resistance(df):
  r1 = df['high'][-20:-5].sort_values(ascending=True)[-2:].mean()
  r2 = df['high'][-60:-20].sort_values(ascending=True)[-2:].mean()
  r3 = df['high'][-100:-60].sort_values(ascending=True)[-2:].mean()
  resistance_list = list([r1, r2, r3])
  s1 = df['low'][-20:-5].sort_values(ascending=False)[-2:].mean()
  s2 = df['low'][-60:-20].sort_values(ascending=False)[-2:].mean()
  s3 = df['low'][-100:-60].sort_values(ascending=False)[-2:].mean()
  list_all = list([s1, s2, s3]) + resistance_list
  list_all.sort()
  print(list_all)
  return list_all[0], list_all[1], list_all[2], list_all[3], list_all[4], list_all[5]

def delta(df):
  df = df[['open', 'high', 'low','close']]
  x = df.to_numpy()
  x = x.flatten()
  min = x.min()
  max = x.max()
  # Compute frequency and bins
  frequency, bins = np.histogram(x, bins=40, range=[min, max])
  dat = pd.DataFrame({'freq': frequency, 'bins': bins[1:]}).sort_values('freq')
  return dat

def rsi_ma(df):
  return ta.sma(ta.rsi(df['close']),5)

def rsi(df):
  df.ta.stochrsi(append=True)
  df.ta.rsi(append=True)
  return df

def madrid_ribbon(df):
  df.ta.ema(5, append=True)
  df.ta.ema(10, append=True)
  df.ta.ema(15, append=True)
  df.ta.ema(20, append=True)
  df.ta.ema(25, append=True)
  df.ta.ema(30, append=True)
  df.ta.ema(35, append=True)
  df.ta.ema(40, append=True)
  df.ta.ema(45, append=True)
  df.ta.ema(50, append=True)
  df.ta.ema(55, append=True)
  df.ta.ema(60, append=True)
  df.ta.ema(65, append=True)
  df.ta.ema(70, append=True)
  df.ta.ema(75, append=True)
  df.ta.ema(80, append=True)
  df.ta.ema(85, append=True)
  df.ta.ema(90, append=True)
  df.ta.ema(95, append=True)
  df.ta.ema(100, append=True)
  df.ta.ema(110, append=True)
  df.ta.ema(120, append=True)
  df.ta.ema(130, append=True)
  df.ta.ema(140, append=True)
  df.ta.ema(150, append=True)
  df.ta.ema(160, append=True)
  return df

def flooring(number, base=1):
    return int(number//base *base)

def trail_sl(current_price, sl, sl_percent, position):
    """ 
    sl_percent for long == 0.93
    sl_percent for sort == 1.07
    """
    if position == 'long':
        if current_price *( sl_percent) > sl:
            return flooring(current_price*(sl_percent))
        else:
            return s1
    if position == 'sort':
        if current_price *( sl_percent) < sl:
            return flooring(current_price*(sl_percent))
        else:
            return sl

def main():
  rm_symbol = ['DIVISLAB']
  symbol1 = list(pd.read_csv('/ind_nifty50list.csv')['Symbol'])
  symbol = [item for item in symbol1 if item not in rm_symbol]
  chat_id = '@stock_chattel' 
  long_symbol = []
  sort_symbol = []
  buy = {}
  sell = {}
  buyprice = {}
  sellprice = {}
  for item in symbol:
      buy[item] = True
      sell[item] =True
      buyprice[item] = 0
      sellprice[item] = 0
  timezone = pytz.timezone('Asia/Kolkata')
  while datetime.now(tz=timezone).time() < time(15, 30, 0):
    #if datetime.now(tz=timezone).time().minute % 2 == 0:
      for item in symbol:
        try:
          sleep(round(np.random.uniform(0.5, 2),2))
          df = yf.download('{}.NS'.format(item), period='2d', interval = '1m',progress=False)
          df.columns = ['open','high','low','close','adj_close','volume']
          df = df[['open','high','low','close','volume']]
          df.reset_index(inplace=True)
          df = squeeze_momentum(df.copy())
          df = rsi(df.copy())
          df = madrid_ribbon(df.copy())
          df['rsi_ma'] = rsi_ma(df.copy())
          df['std'] = df.EMA_10.rolling(10).std()*100/df['close']
          df['avg_close_diff'] = ta.ema((1 - df['EMA_100'] / df['close']),2)
          df.dropna(axis=0,inplace=True)
          df = df.round(decimals=2)
          if df['RSI_14'].iloc[-2:].max() > 75 and  df['RSI_14'].iloc[-1:].values[0] < 75:
                
                if sellprice[item] ==0:
                    sellprice[item] = df['close'].iloc[-1:].values[0]
                else:
                    sellprice[item] = (sellprice[item] + df['close'].iloc[-1:].values[0])/2

                text = "**Intraday** \n Time: {} \n Symbol: {} \n Price: {} \n **Bearish**".format(
                    datetime.now(tz=timezone).strftime('%H:%M:%S'),item, df['close'].iloc[-1:].values[0])
                print(text)
                base_url = "https://api.telegram.org/bot{}/sendMessage?chat_id={}&text={}".format('1242019168:AAGB9EVv01WkskLJf3DlFp7C_dNvwm1r21E', chat_id, text)
                #requests.get(base_url)
                sell[item] = False

          if not sell[item]:
              if df['RSI_14'].iloc[-1:].values[0] > 30 and df['RSI_14'].iloc[-2:-1].values[0] < 30 or datetime.now(tz=timezone).time() > time(15, 24, 0):
                profit = (sellprice[item]/df['close'].iloc[-1:].values[0] - 1) *100
                text = "**Intraday** \n Time: {} \n Symbol: {} \n Price: {} \n Profit: {} \n **Exitting sort**".format(
                    datetime.now(tz=timezone).strftime('%H:%M:%S'),  item, df['close'].iloc[-1:].values[0], profit)
                print(text)
                base_url = "https://api.telegram.org/bot{}/sendMessage?chat_id={}&text={}".format('1242019168:AAGB9EVv01WkskLJf3DlFp7C_dNvwm1r21E', chat_id, text)
                #requests.get(base_url)
                sell[item] = True

          if df['RSI_14'].iloc[-2:].min() < 25 and  df['RSI_14'].iloc[-1:].values[0] > 25:
               
                if buyprice[item] ==0:
                    buyprice[item] = df['close'].iloc[-1:].values[0]
                else:
                    buyprice[item] = (buyprice[item] + df['close'].iloc[-1:].values[0])/2

                text = "**Intraday** \n Time: {} \n Symbol: {} \n Price: {} \n **Bullish**".format(
                    datetime.now(tz=timezone).strftime('%H:%M:%S'),  item, df['close'].iloc[-1:].values[0] )
                print(text)
                base_url = "https://api.telegram.org/bot{}/sendMessage?chat_id={}&text={}".format('1242019168:AAGB9EVv01WkskLJf3DlFp7C_dNvwm1r21E', chat_id, text)
                #requests.get(base_url)
                buy[item] = False
          if not buy[item]:
            if df['RSI_14'].iloc[-1:].values[0] < 70 and df['RSI_14'].iloc[-2:-1].values[0] > 70 or datetime.now(tz=timezone).time() > time(15, 24, 0):
                profit = (df['close'].iloc[-1:].values[0]/buyprice[item] - 1) *100
                text = "**Intraday** \n Time: {} \n Symbol: {} \n Price: {} \n Profit: {} \n **Exitting long**".format(
                    datetime.now(tz=timezone).strftime('%H:%M:%S'),  item, df['close'].iloc[-1:].values[0], profit )
                print(text)
                base_url = "https://api.telegram.org/bot{}/sendMessage?chat_id={}&text={}".format('1242019168:AAGB9EVv01WkskLJf3DlFp7C_dNvwm1r21E', chat_id, text)
                #requests.get(base_url)
                buy[item] = True

        except Exception as e:
          print(e)
          continue

main()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 5.0 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=69a2eddb5b2a09c923d80e0edaef95a8fde63b63cf1e48c8170ec8541081b26f
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 11.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency reso

In [ ]:
 df = yf.download('{}.NS'.format('SBIN'), period='2d', interval = '1m',progress=False)
 df.tail()

NameError: ignored

In [ ]:
df.columns = ['open','high','low','close','adj_close','volume']
df = df[['open','high','low','close','volume']]
df.reset_index(inplace=True)
df = squeeze_momentum(df.copy())
df = rsi(df.copy())
df = madrid_ribbon(df.copy())
df['rsi_ma'] = rsi_ma(df.copy())
df['std'] = df.EMA_10.rolling(10).std()*100/df['close']
df['avg_close_diff'] = ta.ema((1 - df['EMA_100'] / df['close']),2)
df.dropna(axis=0,inplace=True)
df = df.round(decimals=2)
df.tail()

NameError: ignored

In [ ]:
df = yf.download('{}.NS'.format('SBIN'), period='2d', interval = '1m',progress=False)
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-07-28 15:26:00+05:30,532.150024,532.450012,532.049988,532.099976,532.099976,49146
2022-07-28 15:27:00+05:30,532.349976,532.700012,532.099976,532.700012,532.700012,40809
2022-07-28 15:28:00+05:30,532.349976,532.700012,532.099976,532.250000,532.250000,62934
2022-07-28 15:29:00+05:30,532.099976,532.349976,531.900024,532.049988,532.049988,35159
2022-07-28 15:30:00+05:30,532.450012,532.450012,532.450012,532.450012,532.450012,0


In [ ]:
df = yf.ticker('SBIN', period='2d', interval = '1m')
df

TypeError: ignored